In [14]:
# Bibiliotecas;
import pandas as pd 
from IPython.display import clear_output

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [2]:
# Load dados;
data = pd.read_csv("../Databases/autos.csv", encoding="ISO-8859-1")

In [3]:
# Removendo coluna de dados que não estão relacionadas com o veiculo;
data = data.drop(['dateCrawled','name','lastSeen','seller',
                  'postalCode','dateCreated','nrOfPictures', 
                  'offerType'], axis=1)

# Fazendo filtragem de dados;
data = data[(data.price > 100) & (data.price < 350000)]

# Valores nulos, verificando existencia;
data.loc[pd.isnull(data["vehicleType"])] 
data.loc[pd.isnull(data["gearbox"])]
data.loc[pd.isnull(data["model"])] 
data.loc[pd.isnull(data["fuelType"])]
data.loc[pd.isnull(data["notRepairedDamage"])] 

# Dicionario de substituição, baseado no valor que mais se repete;
values = {"vehicleType": data["vehicleType"].value_counts().index[0], # limousine
          "gearbox": data["gearbox"].value_counts().index[0], # manuell
          "model": data["model"].value_counts().index[0], # golf     
          "fuelType": data["fuelType"].value_counts().index[0], # benzin
          "notRepairedDamage": data["notRepairedDamage"].value_counts().index[0]} # nein

# Filtro para valors nulos;
data = data.fillna(values)

In [4]:
# Separando features de label;
X = data.iloc[:,1:13].values
y = data.iloc[:,0].values

# Transformando categoricas para numerico;
categorical = [0,1,3,5,8,9,10]
encoder = LabelEncoder()
for i in categorical:
    X[:,i] = encoder.fit_transform(X[:,i]);

# Transformando pra p formato oneHot
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), categorical)],remainder='passthrough')
X = onehotencorder.fit_transform(X).toarray()


In [22]:
def createNetwork():
    # Criando rede neural;
    regressor = Sequential([Dense(units=158, activation="relu", input_dim=316),
                            Dense(units=158, activation="relu"),
                            Dense(units=1, activation="linear")])

    # Comficurando compile;
    regressor.compile(optimizer="adam",loss="mean_absolute_error", metrics = ["mean_absolute_error"])

    return regressor

In [24]:
# Construindo Modelo;
regressor = KerasRegressor(build_fn=createNetwork,
                        epochs=100,
                        batch_size=300)

# Treinando usando validação cruzada;
result = cross_val_score(estimator=regressor,             # Modelo a ser avaliado
                         X=X, y=y,                        # Dados utilizados;
                         cv=10,                           # Numero de Divisões;
                         scoring="neg_mean_absolute_error") # Tipo de metrica;
clear_output()                        

In [37]:
print("result media: ", result.mean())
print("Desvio: ", result.std())

result mean:  -2335.984200515605
Desvio:  240.80863493831902
